In [0]:
from __future__ import unicode_literals, print_function, division
import math
import operator
import random
from queue import PriorityQueue

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from io import open
import unicodedata
import string
import re
import random
import xml.etree.ElementTree as ET
import os

from torch import optim
import torch.nn.functional as F

In [0]:
 %matplotlib inline

In [0]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [0]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    
    x_files = ['IWSLT14.TED.dev2010.en-fr.en.xml','IWSLT14.TED.tst2010.en-fr.en.xml']
    y_files = ['IWSLT14.TED.dev2010.en-fr.fr.xml','IWSLT14.TED.tst2010.en-fr.fr.xml']
    
    x_files = ['IWSLT14.TED.tst2011.en-fr.en.xml', 'IWSLT14.TED.tst2012.en-fr.en.xml']
    y_files = ['IWSLT14.TED.tst2011.en-fr.fr.xml', 'IWSLT14.TED.tst2012.en-fr.fr.xml']

    eng = []
    fr = []

    for i,j in zip(x_files, y_files):
        tree_eng = ET.parse(i)
        root_eng = tree_eng.getroot()

        tree_fr = ET.parse(j)
        root_fr = tree_fr.getroot()

        for k in root_eng.iter('seg'):
            eng.append(k.text.strip())

        for l in root_fr.iter('seg'):
            fr.append(l.text.strip())
            
    eng_norm = [normalizeString(l) for l in eng]
    fr_norm = [normalizeString(l) for l in fr]
    
    pairs = zip(eng_norm, fr_norm)
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [0]:
MAX_LENGTH = 50

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [33]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 1942 sentence pairs
Trimmed to 59 sentence pairs
Counting words...
Counted words:
fra 374
eng 355
['je vais decouper ici .', 'i m going to make a cut right there .']


In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, n_layers = 1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.n_layers = n_layers

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, n_layers)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers = 1):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers  # 1
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.n_layers = n_layers

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [0]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]])

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
loss_values = []
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            loss_values.append(plot_loss_avg)
            plot_loss_total = 0
    showPlot(plot_losses)

In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

#         decoder_input = torch.tensor([[SOS_token]])  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
        
#         beam_width = 3
        beam_width = 60
        topk = 1  # how many sentence do you want to generate
        decoded_batch = []

        # Start with the start of the sentence token
        decoder_input = torch.LongTensor([SOS_token])

        # Number of sentence to generate
        endnodes = []
        number_required = min((topk + 1), topk - len(endnodes))
        number_required = 1

        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1

        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: 
                break
#             print(nodes.get())
            # fetch the best node
            score, n = nodes.get()
            
            # print('--best node seqs len {} '.format(n.leng))
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue

            # decode for one step using decoder
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)

            # PUT HERE REAL BEAM SEARCH OF TOP
#             print(decoder_output.shape)
            log_prob, indexes = torch.topk(decoder_output, beam_width)
            nextnodes = []

            for new_k in range(beam_width):
                decoded_t = indexes[0][new_k].view(-1)
                log_p = log_prob[0][new_k].item()

#                 print('Previous Node : '.format(n))
                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                # increase qsize
            qsize += len(nextnodes) - 1

        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(topk)]

        utterance = []
        for score, n in sorted(endnodes, key=operator.itemgetter(0)):
            utterance.append(int(n.wordid))
            while n.prevNode != None:
                n = n.prevNode
                utterance.append(int(n.wordid))

        utterance = utterance[::-1]
#                 utterances.append(utterance)

#             decoded_batch.append(utterance)
# #             print(decoded_batch)
#             decoded_batch = decoded_batch[0][0]
        decoded_words = [output_lang.index2word[word] for word in utterance]
        return decoded_words, decoder_attentions     
    

In [0]:
class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward  # Length Penalty: keeping it to 0

    def __lt__(self, other):
        return self.leng < other.leng  

    def __gt__(self, other):
        return self.leng > other.leng

In [44]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000) 
# 2, 37,  3, 19, 18, 10, 11, 22, 38, 12

4m 53s (- 68m 31s) (5000 6%) 1.0494
10m 4s (- 65m 27s) (10000 13%) 0.0074
15m 16s (- 61m 4s) (15000 20%) 0.0033
20m 33s (- 56m 30s) (20000 26%) 0.0021
25m 37s (- 51m 14s) (25000 33%) 0.0015
30m 55s (- 46m 22s) (30000 40%) 0.0012
36m 0s (- 41m 9s) (35000 46%) 0.0010
41m 3s (- 35m 55s) (40000 53%) 0.0008
46m 3s (- 30m 42s) (45000 60%) 0.0007
51m 5s (- 25m 32s) (50000 66%) 0.0007
55m 57s (- 20m 21s) (55000 73%) 0.0006
60m 53s (- 15m 13s) (60000 80%) 0.0005
65m 55s (- 10m 8s) (65000 86%) 0.0005
70m 49s (- 5m 3s) (70000 93%) 0.0004
75m 46s (- 0m 0s) (75000 100%) 0.0004


In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

### Beam width = 3

In [46]:
evaluateRandomly(encoder1, attn_decoder1)

> je suis celui qui ne veut pas etre interrompu .
= i m the one who doesn t want to be interrupted .
< SOS i m the one who doesn t want to be interrupted . EOS

> je vais vous montrer .
= i m going to do some demonstrations .
< SOS i m going to do some demonstrations . EOS

> je vais faire une biennale internationale j ai besoin d artistes du monde entier .
= i m going to do an international biennial i need artists from all around the world .
< SOS i m going to do an international biennial i need artists from all around the world . EOS

> je suis sous l eau dans la rivere avec ces poissons .
= i m under the river with those fish .
< SOS i m under the river with those fish . EOS

> je suis tellement gene que ca ait coute si peu d obtenir que mon idee me soit implantee que je ne suis pas pret a vous dire ce que ca a coute .
= i m so embarrassed at how cheap it was to get from my idea to me being implanted that i m not prepared to tell you what it cost .
< SOS i m so embarrassed at how ch

In [48]:
pip install sacrebleu

     |████████████████████████████████| 61kB 8.4MB/s 


In [0]:
import sacrebleu
def bleuScore(test_y_pred, test_y_actuals): #2 lists
    bleu = sacrebleu.corpus_bleu(test_y_pred, [test_y_actuals])
    return bleu.score

In [0]:
def readLangs_test(lang1, lang2, reverse=False):
    print("Reading lines...")

    x_files = ['IWSLT14.TED.tst2011.en-fr.en.xml', 'IWSLT14.TED.tst2012.en-fr.en.xml']
    y_files = ['IWSLT14.TED.tst2011.en-fr.fr.xml', 'IWSLT14.TED.tst2012.en-fr.fr.xml']
    eng = []
    fr = []

    for i,j in zip(x_files, y_files):
        tree_eng = ET.parse(i)
        root_eng = tree_eng.getroot()

        tree_fr = ET.parse(j)
        root_fr = tree_fr.getroot()

        for k in root_eng.iter('seg'):
            eng.append(k.text.strip())

        for l in root_fr.iter('seg'):
            fr.append(l.text.strip())
            
    eng_norm = [normalizeString(l) for l in eng]
    fr_norm = [normalizeString(l) for l in fr]
    
    pairs = zip(eng_norm, fr_norm)
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [0]:
def evaluateRandomly_test(encoder, decoder, n=10):
    actual_sen = []
    pred_sen = []
    for pair in pairs:
        actual_sen.append(pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        pred_sen.append(output_sentence)
        
    return bleuScore(pred_sen, actual_sen)

In [52]:
evaluateRandomly_test(encoder1, attn_decoder1)

86.96629409321952

### Beam Width = 10

In [57]:
evaluateRandomly(encoder1, attn_decoder1)

> nous commencons a vendre des voitures electriques et c est genial .
= we re starting to sell electric cars which is great .
< SOS we re starting to sell electric cars which is great . EOS

> je suis ingenieur procede .
= i m a process engineer .
< SOS i m a process engineer . EOS

> nous sommes vous etes la ressource la plus sous employee dans le systeme de sante .
= we are you are the most underused resource in health care .
< SOS we are you are the most underused resource in health care . EOS

> il s interesse a la facon dont nous voyons comme un animal combien nous sommes interesses par le mimetisme et le camouflage .
= he is interested in how we see as an animal how we are interested in mimicry and camouflage .
< SOS he is interested in how we see as an animal how we are interested in mimicry and camouflage . EOS

> je vais l organiser et la diriger et l ouvrir au monde .
= i m going to organize it and direct it and get it going in the world .
< SOS i m going to organize it and d

In [58]:
evaluateRandomly_test(encoder1, attn_decoder1)

86.96629409321952

### Beam Width = 30

In [60]:
evaluateRandomly(encoder1, attn_decoder1)

> je vais decouper des veines des arteres oups ! . . .
= i m going to cut some veins arteries . oops ! . . .
< SOS i m going to cut some veins arteries . oops ! . . . EOS

> vous etes pousses a faire quelque chose que tout le monde a juge impossible .
= you re driven to do something that everyone has told you is impossible .
< SOS you re driven to do something that everyone has told you is impossible . EOS

> ils s entrainent juste .
= they re just practicing .
< SOS they re just practicing . EOS

> je suis acteur .
= i m a performer .
< SOS i m a performer . EOS

> je vais me contenter de tout faire moi meme .
= i m just going to make the whole thing myself .
< SOS i m just going to make the whole thing myself . EOS

> nous sommes tous des specialistes maintenant meme les medecins de premiers secours .
= we re all specialists now even the primary care physicians .
< SOS we re all specialists now even the primary care physicians . EOS

> nous sommes subjugues par la technologie .
= we 

In [61]:
evaluateRandomly_test(encoder1, attn_decoder1)

86.96629409321952

### Impact on results

In [0]:
def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))

In [63]:
evaluateAndShowAttention("nous sommes optimiste par la chirurgie .") #beam width = 3

input = nous sommes optimiste par la chirurgie .
output = SOS i m going to do some cuts . EOS


In [66]:
evaluateAndShowAttention("nous sommes optimiste par la chirurgie .") #beam width = 10

input = nous sommes optimiste par la chirurgie .
output = SOS i m going to do some cuts . EOS


In [64]:
evaluateAndShowAttention("nous sommes optimiste par la chirurgie .") #beam width = 30

input = nous sommes optimiste par la chirurgie .
output = SOS i m going to do some cuts . EOS


### Beam Width = 60

In [68]:
evaluateRandomly(encoder1, attn_decoder1)

> ils acceptent plus la tristesse que les gens plus jeunes .
= they re more accepting of sadness than younger people are .
< SOS they re more accepting of sadness than younger people are . EOS

> nous sommes subjugues par la technologie .
= we re smitten with technology .
< SOS we re smitten with technology . EOS

> nous poussons l efficience du carburant vers de nouveaux sommets .
= we re pushing fuel efficiency to new heights .
< SOS we re pushing fuel efficiency to new heights . EOS

> c est une artiste suisse tres serieuse .
= she s a very serious swiss artist .
< SOS she s a very serious swiss artist . EOS

> ils sont aussi assez riches et aises et toutes ces autres sortes de choses .
= they re also fairly wealthy and affluent and all these other sorts of things .
< SOS they re also fairly wealthy and affluent and all these other sorts of things . EOS

> ce sont des specimens de papillons qui ont un camouflage naturel .
= they re butterfly specimens who have a natural camouflage .

In [69]:
evaluateRandomly_test(encoder1, attn_decoder1)

86.99946291478088